##**PROJETO FINAL 02: Construção de um RAG utilizando LangChain**

## **CESAR SCHOOL**
* Pós-graduação em Engenharia e Análise de Dados - 2023.2
* **Disciplina: Tópicos Complementares**
* Professor: **Silvan Ferreira**
* Aluno: **Allan Bispo** - apsb@cesar.school

###**OBJETIVO: Construção de um RAG utilizando LangChain**
* Desenvolvimento de um sistema `RAG (Retrieval-Augmented Generation)` utilizando a `biblioteca LangChain`.
* Aspectos:
  * Escolha do Documento: um ou mais documentos, podendo ser PDF, texto, páginas da web etc;
  * Splitting do Documento;
  * Criação de Vector Store;
  * Retrieval;
  * Geração de Respostas.

###**Apresentação dos Datasets**
#### Para fins de estudo foram explorados 2 fontes de dados sobre o tema Bolo de Rolo.
#### O primeiro serviu para particar o loading de dados no formato PDF, enquanto o segundo o loading de dados no formato HTTP.
  1. Arquivo de texto salvo no formanto PDF que aborda a cultura e hostória do bolo de rolo pernambucano. Texto extraído do site abaixo:
    - https://sites.google.com/etfbsb.edu.br/ahoradocha/projeto-integrador/bolo-de-rolo
  2. Website do GE que conta a historia do Santa Cruz e o bolo de rolo:
    - https://ge.globo.com/pe/futebol/times/santa-cruz/noticia/santa-cruz-inova-na-captacao-de-recursos-e-vende-bolo-de-rolo-para-construir-ct.ghtml

#### **Importando as bibliotecas e API KEY**

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [2]:
!pip install lambdata langchain_community langchain_openai langchain_text_splitters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.

In [3]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 2.0 MB/s eta 0:00:00


In [4]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 46.0 MB/s eta 0:00:00


In [5]:
import bs4
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.document_loaders import PyPDFLoader
from langchain.document_loaders import WebBaseLoader

####**Loading e Splitting - PDF**
* O código carrega o conteúdo de um arquivo PDF usando `PyPDFLoader`, preparando o texto para análise posterior.
* Então divide o texto em segmentos de 1000 caracteres, com sobreposição de 200, usando `RecursiveCharacterTextSplitter`.




In [40]:
# Carregar o documento PDF
file_path_pdf = "/content/bolo_de_rolo.pdf"
loader_pdf = PyPDFLoader(file_path_pdf)
docs_pdf = loader_pdf.load()

In [41]:
# Dividir o texto em chunks
text_splitter_pdf = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True)
all_splits_pdf = text_splitter_pdf.split_documents(docs_pdf)

####**Criação de Vector Store**
* Cria vetores de `embeddings`, que são representações numéricas do texto, e os armazena em um índice de busca `FAISS`, uma biblioteca para busca rápida em grandes volumes de dados.

In [42]:
# Configurar o vetor de recuperação
embeddings = OpenAIEmbeddings()
vectorstore_pdf = FAISS.from_documents(all_splits_pdf, embeddings)

####**Retrieval**
* Utilizar `FAISS` para configurar um `retriever`, uma ferramenta que busca documentos por similaridade, retornando via `invoke` os 3 mais relevantes para a `query` desejada.

In [43]:
# Consultar o vetor de recuperação
retriever_pdf = vectorstore_pdf.as_retriever(search_type="similarity", search_kwargs={"k": 3})

query_pdf = "O bolo de rolo é baiano?"
retrieved_docs_pdf = retriever_pdf.invoke(query_pdf)

####**Definições dos templates e prompts**

In [44]:
# Definindo o template do prompt para o assistente
system_template_pdf = """Você é um assistente para tarefas de perguntas e respostas sobre culinária, receitas e histórias. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases, mantenha a resposta concisa e didática, e fale apenas o necessário, não inclua na sua resposta a frase da pergunta respectiva.
Pergunta: {question}
Contexto: {context}
Resposta:
"""
prompt_template_pdf = ChatPromptTemplate.from_template(system_template_pdf)

In [45]:
example_messages_pdf = prompt_template_pdf.invoke({
    "context": "algum contexto",
    "question": "alguma pergunta"
})

print(example_messages_pdf.to_messages())

[HumanMessage(content='Você é um assistente para tarefas de perguntas e respostas sobre culinária, receitas e histórias. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases, mantenha a resposta concisa e didática, e fale apenas o necessário, não inclua na sua resposta a frase da pergunta respectiva.\nPergunta: alguma pergunta\nContexto: algum contexto\nResposta:\n')]


####**Instanciando o modelo**

In [46]:
llm_pdf = ChatOpenAI(model="gpt-4o-mini")

####**Formatação de Documentos Recuperados**
* Função que formata documentos recuperados, concatenando o conteúdo de cada página com quebras de linha duplas para facilitar a leitura.

In [47]:
# Função para formatar os documentos recuperados
def format_docs_pdf(docs_pdf):
    return "\n\n".join(doc.page_content for doc in docs_pdf)

####**Geração de Respostas**
* Configurar uma `cadeia de Recuperação e Geração (RAG)` que utiliza `retriever_pdf` para buscar e formatar documentos, `prompt_template_pdf` para criar a pergunta, e `llm_pdf` para gerar a resposta.
* O `RunnablePassthrough` passa diretamente os dados entre as etapas, enquanto `StrOutputParser` analisa e formata a resposta gerada pelo modelo.

In [48]:
# Configurando a cadeia de RAG
rag_chain_pdf = (
    {"context": retriever_pdf | format_docs_pdf, "question": RunnablePassthrough()}
    | prompt_template_pdf
    | llm_pdf
    | StrOutputParser())

####**Lista de perguntas**

In [49]:
perguntas_pdf = ['Com base no conteúdo, o que significa o bolo de rolo ser finalizado com açúcar de confeiteiro polvilhado por cima?',
'O bolo de rolo foi criado no Brasil por imigrantes franceses, de acordo?',
'Com base no conteúdo, explique por que o bolo de rolo é considerado uma adaptação do bolo português.',
'A massa do bolo de rolo deve ser assada em camadas finas para facilitar o enrolamento?',
'O bolo de rolo é tradicionalmente assado em formas redondas e altas?',
'A origem do bolo de rolo remonta ao período colonial, quando foi trazido pelos holandeses, está correto?'
]

####**Executando a RAG chain**
* Iterar sobre uma `lista de perguntas`, executa a `cadeia RAG` para cada uma, e imprime as respostas geradas. Cada `chunk` é uma parte da resposta completa exibida em tempo real.

In [50]:
# Iterando sobre cada pergunta e executando a cadeia de RAG para cada uma
for i, pergunta in enumerate(perguntas_pdf):
    print(f"Pergunta {i+1}: {pergunta}")
    for chunk in rag_chain_pdf.stream(pergunta):
        print(chunk, end="", flush=True)
    print("\n")

Pergunta 1: Com base no conteúdo, o que significa o bolo de rolo ser finalizado com açúcar de confeiteiro polvilhado por cima?
Finalizar o bolo de rolo com açúcar de confeiteiro polvilhado por cima acrescenta um toque doce e decorativo, realçando a apresentação do doce. Essa finalização também complementa o sabor do recheio de goiaba, tornando a experiência ainda mais agradável.

Pergunta 2: O bolo de rolo foi criado no Brasil por imigrantes franceses, de acordo?
O bolo de rolo é uma adaptação do "colchão de noiva", um doce de origem portuguesa, e foi introduzido no Brasil pelos colonizadores. Portanto, não foi criado por imigrantes franceses, mas sim pelos portugueses.

Pergunta 3: Com base no conteúdo, explique por que o bolo de rolo é considerado uma adaptação do bolo português.
O bolo de rolo é considerado uma adaptação do bolo português porque ele deriva do "colchão de noiva", um pão de ló enrolado, que foi modificado ao trocar o recheio de nozes pela goiaba, uma fruta abundante n

------------------------------------------------------
####**Loading e Splitting - HTTP**
* Utilizar `BeautifulSoup` com `SoupStrainer` para `filtrar o HTML` e extrair somente o conteúdo da `classe mc-article-body`. Em seguida, o` WebBaseLoader` carrega a página web especificada e aplica o filtro para obter e armazenar o texto relevante em `docs_web`.
* Então divide o texto em segmentos de 1000 caracteres, com sobreposição de 200, usando `RecursiveCharacterTextSplitter`.

In [17]:
!pip install beautifulsoup4
!pip install langchain

In [51]:
# Filtra o conteúdo da página por uma classe específica
bs4_strainer = bs4.SoupStrainer(class_=("mc-article-body"))

# Carrega o conteúdo da página
loader_web = WebBaseLoader(
    web_paths=("https://ge.globo.com/pe/futebol/times/santa-cruz/noticia/santa-cruz-inova-na-captacao-de-recursos-e-vende-bolo-de-rolo-para-construir-ct.ghtml",),
    bs_kwargs={"parse_only": bs4_strainer},
)

# Carrega o conteúdo da página
docs_web = loader_web.load()

In [52]:
text_splitter_web = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits_web = text_splitter_web.split_documents(docs_web)

####**Criação de Vector Store, Retrieval, templates e prompts**
* O mesmo do processo com PDF.

In [53]:
vectorstore_web = FAISS.from_documents(all_splits_web, OpenAIEmbeddings())

In [54]:
retriever_web = vectorstore_web.as_retriever(search_type="similarity", search_kwargs={"k": 3})

retrieved_docs_web = retriever_web.invoke("Qual o time de futebol que obteve ajuda via bolo de rolo?")

In [55]:
# Definindo o template do prompt para o assistente
system_template_web = """Você é um assistente para tarefas de perguntas e respostas sobre futebol, culinária, receitas e histórias. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases, mantenha a resposta concisa e didática, e fale apenas o necessário, não inclua na sua resposta a frase da pergunta respectiva.
Pergunta: {question}
Contexto: {context}
Resposta:
"""
prompt_template_web = ChatPromptTemplate.from_template(system_template_web)

In [56]:
example_messages_web = prompt_template_web.invoke({
    "context": "algum contexto",
    "question": "alguma pergunta"
})

print(example_messages_web.to_messages())

[HumanMessage(content='Você é um assistente para tarefas de perguntas e respostas sobre futebol, culinária, receitas e histórias. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases, mantenha a resposta concisa e didática, e fale apenas o necessário, não inclua na sua resposta a frase da pergunta respectiva.\nPergunta: alguma pergunta\nContexto: algum contexto\nResposta:\n')]


####**Escolha do modelo, formatação do texto e RAG Chain**
* O mesmo do processo com PDF.

In [57]:
llm_web = ChatOpenAI(model="gpt-4o-mini")

In [58]:
def format_docs(docs_web):
    return "\n\n".join(doc.page_content for doc in docs_web)

In [59]:
rag_chain_web = (
    {"context": retriever_web | format_docs, "question": RunnablePassthrough()}
    | prompt_template_web
    | llm_web
    | StrOutputParser()
)

####**Lista de perguntas - HTTP**

In [60]:
perguntas_web = ['Qual time de futebol que obteve ajuda via bolo de rolo?',
'O Sport Clube do Recife também adotou a venda de bolo de rolo para ajudar na construção do seu centro de treinamento, correto?',
'De acordo com o conteúdo do site, explique o motivo do Santa Cruz vender bolo de rolo.'
]

####**Execução da RAG Chain**
* O mesmo do processo com PDF.

In [61]:
# Iterando sobre cada pergunta e executando a cadeia de RAG para cada uma
for i, pergunta in enumerate(perguntas_web):
    print(f"Pergunta {i+1}: {pergunta}")
    for chunk in rag_chain_web.stream(pergunta):
        print(chunk, end="", flush=True)
    print("\n")

Pergunta 1: Qual time de futebol que obteve ajuda via bolo de rolo?
O time de futebol que obteve ajuda via bolo de rolo é o Santa Cruz. O clube vende o produto para arrecadar fundos para a construção do seu Centro de Treinamento.

Pergunta 2: O Sport Clube do Recife também adotou a venda de bolo de rolo para ajudar na construção do seu centro de treinamento, correto?
Não sei.

Pergunta 3: De acordo com o conteúdo do site, explique o motivo do Santa Cruz vender bolo de rolo.
O Santa Cruz está vendendo bolo de rolo para arrecadar fundos para a construção de seu Centro de Treinamento, utilizando a receita para minimizar o tempo de espera até a conclusão das obras. A ideia surgiu como uma forma criativa de engajar os torcedores e facilitar a contribuição para o projeto.



####**CONCLUSÃO RAG CHAIN**:

* Conforme visto acima, tanto com dados oriundos de um arquivo PDF quanto de um web site, passado a fase de loading e splitting, que são caracteristicas de cada padrão de dados de entrada, o processo de construção do encadeamento da RAG é similar.
* As perguntas foram elaboradas para estressar o modelo e, quando o mesmo não soube da resposta, não alucionou, respondeu como deveria 'Não sei'.